In [43]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, ExactLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.qfts import Standard as StandardQFTS
from qiskit.aqua.components.iqfts import Standard as StandardIQFTS
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

c:\users\hrush\.conda\envs\quantum_computing\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:181: UserWarning: Credentials are already in use. The existing account in the session will be replaced.
  warnings.warn('Credentials are already in use. The existing '


In [44]:
def create_eigs(matrix, num_ancillae, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [StandardQFTS(num_ancillae - 1), StandardIQFTS(num_ancillae - 1)]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   StandardIQFTS(num_ancillae),
                   num_time_slices=50,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

def fidelity(hhl, ref):
    solution_hhl_normed = hhl / np.linalg.norm(hhl)
    solution_ref_normed = ref / np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed)
    print("fidelity %f" % fidelity)

In [45]:
# set the random seed to get the same pseudo-random matrix for every run
np.random.seed(1)
matrix = a = [[11.5384615384615,-4.80769230769231,2.17948717948718,-0.961538461538462],
     [-4.80769230769231,8.84615384615385,-0.961538461538462,1.69230769230769],
     [-2.30769230769231,0.961538461538462,10.8974358974359,-4.80769230769231],
     [0.961538461538462,-1.76923076923077,-4.80769230769231,8.46153846153846]]
vector = [-0.53589744, -0.21153846, 0.02051282, 0.09230769]

print("random matrix:")
m = np.array(matrix)
#print(np.round(m, 3))

orig_size = len(vector)
matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrix, vector)
print("random matrix:")
# Initialize eigenvalue finding module
eigs = create_eigs(matrix, 6, True)
num_q, num_a = eigs.get_register_sizes()
print("random matrix2:")
# Initialize initial state module
init_state = Custom(num_q, state_vector=vector)
print("random matrix3:")
# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)
print("random matrix4:")
algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)
result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))

print("solution ", np.round(result['solution'], 5))

result_ref = ExactLSsolver(matrix, vector).run()
print("classical solution ", np.round(result_ref['solution'], 5))
result = algo.run(QuantumInstance(BasicAer.get_backend('qasm_simulator')))
print("solution sampling method ", np.round(result['solution'], 5))
#print("probability %f" % result['probability_result'])
#fidelity(result['solution'], result_ref['solution'])

random matrix:


AttributeError: type object 'HHL' has no attribute 'matrix_resize'